In [1]:
start = "2024-01-01"
end = "2024-03-01"
interval = "1h"
tickers = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "ADAUSDT", "AVAXUSDT", "DOGEUSDT", "TRXUSDT",
           "DOTUSDT", "LINKUSDT", "SHIBUSDT", "LTCUSDT", "BCHUSDT", "UNIUSDT"]

In [2]:
from modules.data_services.data_pipeline import load_data

# 1. Load data
df = load_data(
    tickers=tickers,
    start=start,
    end=end,
    interval=interval,
)
print(df.head())

                      BTCUSDT  ETHUSDT  BNBUSDT  SOLUSDT  XRPUSDT  ADAUSDT  \
open_time                                                                    
2024-01-01 00:00:00  42475.23  2295.51    314.4   101.96   0.6162   0.5979   
2024-01-01 01:00:00  42613.56  2303.72    315.3   104.12   0.6185   0.6023   
2024-01-01 02:00:00  42581.10  2293.02    310.9   103.69   0.6154   0.5995   
2024-01-01 03:00:00  42330.49  2273.81    309.2   103.07   0.6130   0.5956   
2024-01-01 04:00:00  42399.99  2279.55    309.2   102.62   0.6116   0.5953   

                     AVAXUSDT  DOGEUSDT  TRXUSDT  DOTUSDT  LINKUSDT  SHIBUSDT  \
open_time                                                                       
2024-01-01 00:00:00     38.94   0.08983  0.10832    8.267    15.122   0.00001   
2024-01-01 01:00:00     39.39   0.09016  0.10820    8.294    15.167   0.00001   
2024-01-01 02:00:00     39.04   0.08980  0.10781    8.253    15.065   0.00001   
2024-01-01 03:00:00     38.51   0.08921  0.10711

In [4]:
# 1. Calculate a sum of standard deviation

from modules.pair_selection.statistical_tests import calculate_ssd_of_cumulative_returns

ssd_df = calculate_ssd_of_cumulative_returns(df).sort_values('ssd_c_returns', ascending=True).reset_index(drop=True)
print(ssd_df.head())

                pair  ssd_c_returns
0  DOGEUSDT-SHIBUSDT       1.316605
1    XRPUSDT-DOTUSDT       2.393193
2    ADAUSDT-DOTUSDT       2.591386
3   SHIBUSDT-LTCUSDT       3.702435
4   ADAUSDT-AVAXUSDT       3.805218


In [ ]:
# 2. Load chosen pair data

from modules.data_services.data_pipeline import load_pair

pair = ssd_df['pair'][2].split('-')
x = pair[0]
y = pair[1]

start = "2024-03-01"
end = "2024-03-08"

pair = load_pair(
    x=x,
    y=y,
    start=start,
    end=end,
    interval=interval
)
print(pair.data.head())

In [ ]:
# 3. Normalize prices

from modules.data_services.data_pipeline import normalize_with_cumulative_returns
from modules.visualization.plots import plot_prices

normalize_with_cumulative_returns(pair)
plot_prices(pair)
print(pair.data.head())

In [ ]:
# 4. Calculate rolling z-score

from modules.data_services.data_pipeline import calculate_rolling_zscore
from modules.visualization.plots import plot_zscore

calculate_rolling_zscore(pair)
plot_zscore(pair)

In [ ]:
# 5. Generate action space

from modules.performance.strategy import generate_action_space
from modules.visualization.plots import plot_zscore_with_thresholds

entry_threshold = 2
exit_threshold = 0
position_size = 1

generate_action_space(
    pair_data=pair,
    entry_threshold=entry_threshold,
    exit_threshold=exit_threshold,
    position_size=position_size
)
print(pair.data.head())
plot_zscore_with_thresholds(pair)

In [ ]:
# 6. Benchmark strategy

from modules.performance.strategy import benchmark_strategy

initial_cash = 100000
fee_rate = 0.0005

benchmark_strategy(
    pair_data=pair,
    initial_cash=initial_cash,
    fee_rate=fee_rate
)
print(pair.data)

In [ ]:
from modules.visualization.plots import plot_pnl, plot_positions

plot_pnl(pair)
plot_positions(pair)

In [ ]:
from modules.performance.strategy import calculate_stats

stats = calculate_stats(pair)
print(stats)